<a href="https://colab.research.google.com/github/ullasbc02/obesity-risk-analytics/blob/main/07_forecasting_2030.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
from sklearn.linear_model import LinearRegression
import json
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
PATH = '/content/drive/MyDrive/obesity-analytics-notebooks/Multi-Year-Trend/Dataset'
df = pd.read_csv(
    os.path.join(PATH, 'final_clean_county_panel_2010_2023.csv'),
    dtype={"geoId": str}
)
df["fips"] = df["fips"].astype(str).str.zfill(5)
df

,geoid,year,county_name,state_abbr,metro_nonmetro,obesity_rate,poverty_rate,physical_inactivity,median_household_income,unemployment_rate,fips
0,us-ct-001,2010,Fairfield County,CT,Metropolitan,0.183000,0.093000,0.197000,74634.0,0.088472,09001
1,us-ct-001,2011,Fairfield County,CT,Metropolitan,0.191000,0.094000,0.205000,77065.0,0.083891,09001
2,us-ct-001,2012,Fairfield County,CT,Metropolitan,0.206000,0.089000,0.197000,79536.0,0.077980,09001
3,us-ct-001,2013,Fairfield County,CT,Metropolitan,0.211000,0.096000,0.208000,81816.0,0.074060,09001
4,us-ct-001,2014,Fairfield County,CT,Metropolitan,0.218000,0.090000,0.189000,85336.0,0.061873,09001
...,...,...,...,...,...,...,...,...,...,...,...
43983,us-ct-160,2019,Northwest Hills Planning Region,CT,Nonmetropolitan,0.306006,0.134006,0.246007,53366.5,0.036933,09160
43984,us-ct-160,2020,Northwest Hills Planning Region,CT,Nonmetropolitan,0.308004,0.128002,0.227007,55153.0,0.065901,09160
43985,us-ct-160,2021,Northwest Hills Planning Region,CT,Nonmetropolitan,0.315006,0.136000,0.222020,56632.0,0.044242,09160
43986,us-ct-160,2022,Northwest Hills Planning Region,CT,Nonmetropolitan,0.278002,0.079001,0.186000,87206.0,0.034412,09160


In [ ]:
df = df.sort_values(["fips", "year"]).reset_index(drop=True)

In [ ]:
df["obesity_lag1"] = df.groupby("fips")["obesity_rate"].shift(1)
df["obesity_lag2"] = df.groupby("fips")["obesity_rate"].shift(2)

df["inactivity_lag1"] = df.groupby("fips")["physical_inactivity"].shift(1)
df["poverty_lag1"] = df.groupby("fips")["poverty_rate"].shift(1)
df["income_lag1"] = df.groupby("fips")["median_household_income"].shift(1)
df["unemployment_lag1"] = df.groupby("fips")["unemployment_rate"].shift(1)

In [ ]:
df["obesity_next"] = df.groupby("geoid")["obesity_rate"].shift(-1)

In [ ]:
df_model = df.dropna()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error

features = [
    "obesity_lag1", "obesity_lag2",
    "inactivity_lag1", "poverty_lag1",
    "income_lag1", "unemployment_lag1"
]

X = df_model[features]
y = df_model["obesity_next"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = GradientBoostingRegressor(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("R2:", r2_score(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred))


R2: 0.6610956754062964
RMSE: 0.0005242885665067998


In [ ]:
# Ensure sorted and complete
df = df.sort_values(["geoid", "year"])

last_obs_year = 2023
target_years = list(range(2024, 2031))  # 2024–2030 inclusive

# Pivot obesity history: columns = years, rows = counties
obesity_hist = df.pivot(index="geoid", columns="year", values="obesity_rate")

# Drivers fixed at 2023 levels
drivers_2023 = (
    df[df["year"] == last_obs_year]
    .set_index("geoid")[[
        "physical_inactivity",
        "poverty_rate",
        "median_household_income",
        "unemployment_rate",
        "county_name",
        "state_abbr"
    ]]
)

In [ ]:
forecast_rows = []

for year in target_years:
    lag1_year = year - 1
    lag2_year = year - 2

    # Build feature matrix for all counties for this forecast year
    feat = pd.DataFrame(index=drivers_2023.index)

    # Lags from obesity history (actual or previously predicted)
    feat["obesity_lag1"] = obesity_hist[lag1_year]
    feat["obesity_lag2"] = obesity_hist[lag2_year]

    # Drivers held at 2023 levels
    feat["inactivity_lag1"] = drivers_2023["physical_inactivity"]
    feat["poverty_lag1"] = drivers_2023["poverty_rate"]
    feat["income_lag1"] = drivers_2023["median_household_income"]
    feat["unemployment_lag1"] = drivers_2023["unemployment_rate"]

    # Predict next year's obesity
    y_forecast = model.predict(feat[features])

    # Optional: clip to [0, 1] to keep as valid rates
    y_forecast = np.clip(y_forecast, 0, 1)

    # Store into obesity_hist so later years can use as lag
    obesity_hist[year] = y_forecast

    # Build nice forecast dataframe for this year
    tmp = pd.DataFrame({
        "geoid": feat.index,
        "year": year,
        "obesity_rate_pred": y_forecast
    })

    # Add county + state for readability
    tmp = tmp.join(drivers_2023[["county_name", "state_abbr"]], on="geoid")

    forecast_rows.append(tmp)

# Combine all years
df_forecasts = pd.concat(forecast_rows, ignore_index=True)

In [ ]:
# 2023 actual
base_2023 = (
    df[df["year"] == 2023][["geoid", "county_name", "state_abbr", "obesity_rate"]]
    .rename(columns={"obesity_rate": "obesity_2023"})
)

# 2030 forecast
pred_2030 = (
    df_forecasts[df_forecasts["year"] == 2030][["geoid", "obesity_rate_pred"]]
    .rename(columns={"obesity_rate_pred": "obesity_2030_pred"})
)

summary_2030 = base_2023.merge(pred_2030, on="geoid", how="inner")
summary_2030["change_2023_to_2030"] = summary_2030["obesity_2030_pred"] - summary_2030["obesity_2023"]

summary_2030.head()

,geoid,county_name,state_abbr,obesity_2023,obesity_2030_pred,change_2023_to_2030
0,us-ak-013,Aleutians East Borough,AK,0.272066,0.306325,0.034259
1,us-ak-110,Juneau City And Borough,AK,0.307010,0.323519,0.016509
2,us-ak-122,Kenai Peninsula Borough,AK,0.301002,0.332608,0.031606
3,us-ak-130,Ketchikan Gateway Borough,AK,0.375009,0.355203,-0.019806
4,us-ak-150,Kodiak Island Borough,AK,0.301020,0.333919,0.032899


In [ ]:
summary_2030["change_2023_to_2030"].describe()

,change_2023_to_2030
count,3142.000000
mean,0.020710
std,0.025222
min,-0.059174
25%,0.002965
50%,0.019574
75%,0.039503
max,0.101503


In [ ]:
# Top 10 worst projected increases
summary_2030.sort_values("change_2023_to_2030", ascending=False).head(10)

,geoid,county_name,state_abbr,obesity_2023,obesity_2030_pred,change_2023_to_2030
531,us-ga-293,Upson County,GA,0.258000,0.359503,0.101503
1614,us-mt-033,Garfield County,MT,0.219016,0.309818,0.090803
1543,us-ms-055,Issaquena County,MS,0.225071,0.315650,0.090580
1637,us-mt-079,Prairie County,MT,0.215186,0.304752,0.089566
363,us-fl-087,Monroe County,FL,0.227000,0.314627,0.087627
1151,us-la-077,Pointe Coupee Parish,LA,0.243000,0.330354,0.087354
263,us-co-055,Huerfano County,CO,0.222026,0.306642,0.084616
361,us-fl-085,Martin County,FL,0.198001,0.281494,0.083493
1217,us-me-005,Cumberland County,ME,0.248000,0.331146,0.083146
2021,us-ny-081,Queens County,NY,0.254000,0.335806,0.081806


In [ ]:
df_forecasts.to_csv("county_obesity_forecasts_2024_2030.csv", index=False)